# 実験の実行

Azure Machine Learning SDKを使用して、メトリックを記録して出力を生成するコード実験を実行できます。これは、Azure Machine Learningのほとんどの機械学習操作の中核です。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 実験を行う

データサイエンティストが実行する必要がある最も基本的なタスクの1つは、データを処理および分析する実験を作成して実行することです。この演習では、Azure ML *experiment*を使用してPythonコードを実行し、データから抽出された値を記録する方法を学習します。この場合、糖尿病の検査を受けた患者の詳細を含む簡単なデータセットを使用します。実験を実行して、データを探索し、統計、視覚化、およびデータサンプルを抽出します。使用するコードのほとんどは、データ探索プロセスで実行できるような、かなり汎用的なPythonです。ただし、数行を追加すると、コードはAzure ML *experiment*を使用して実行の詳細をログに記録します。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace = ws, name = "diabetes-experiment")

# Start logging data from the experiment
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the data from a local file
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Plot and log the count of diabetic vs non-diabetic patients
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name = 'label distribution', plot = fig)

# log distinct pregnancy counts
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# Log summary statistics for numeric columns
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat = keys[index], value = values[index])
        
# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name = 'outputs/sample.csv', path_or_stream = './sample.csv')

# Complete the run
run.complete()

## 実験結果を見る

実験が終了したら、**run**オブジェクトを使用して、実行とその出力に関する情報を取得できます。:

In [ ]:
import json

# Get run details
details = run.get_details()
print(details)

# Get logged metrics
metrics = run.get_metrics()
print(json.dumps(metrics, indent=2))

# Get output files
files = run.get_file_names()
print(json.dumps(files, indent=2))

Jupyter Notebooksでは、**RunDetails**ウィジェットを使用して、実行の詳細を視覚化できます。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

Note that the **RunDetails** widget includes a link to view the run in Azure Machine Learning studio. Click this to open a new browser tab with the run details (you can also just open [Azure Machine Learning studio](https://ml.azure.com) and find the run on the **Experiments** page). When viewing the run in Azure Machine Learning studio, note the following:

- The **Properties** tab contains the general properties of the experiment run.
- The **Metrics** tab enables you to select logged metrics and view them as tables or charts.
- The **Images** tab enables you to select and view any images or plots that were logged in the experiment (in this case, the *Label Distribution* plot)
- The **Child Runs** tab lists any child runs (in this experiment there are none).
- The **Outputs** tab shows the output files generated by the experiment.
- The **Logs** tab shows any logs that were generated by the compute context for the experiment (in this case, the experiment was run inline so there are no logs).
- The **Snapshots** tab contains all files in the folder where the experiment code was run (in this case, everything in the same folder as this notebook).
- The **Raw JSON** tab shows a JSON representation of the experiment details.
- The **Explanations** tab is used to show model explanations generated by the experiment (in this case, there are none).

## 実験スクリプトを実行する

前の例では、このノートブックで実験をインラインで実行しました。より柔軟なソリューションは、実験用に個別のスクリプトを作成し、必要な他のファイルとともにフォルダーに保存し、Azure MLを使用してフォルダー内のスクリプトに基づいて実験を実行することです。

まず、実験ファイル用のフォルダーを作成し、そこにデータをコピーしましょう:

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

次に、実験用のコードを含むPythonスクリプトを作成し、実験フォルダーに保存します。

> **Note**: 次のセルを実行すると、スクリプトファイルが「作成」されるだけです-実行されません！

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# Save a sample of the data in the outputs folder (which gets uploaded automatically)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

このコードは、以前に使用されていたインラインコードの簡易バージョンです。ただし、次のことに注意してください。
- `Run.get_context（）`メソッドを使用して、スクリプトの実行時に実験実行コンテキストを取得します。
- スクリプトが存在するフォルダーから糖尿病データをロードします。
- **outputs**という名前のフォルダーを作成し、そこにサンプルファイルを書き込みます-このフォルダーは自動的に実験実行にアップロードされます

これで、実験を実行する準備がほぼ整いました。対処する必要がある設定の問題がいくつかあります:

1. スクリプトのPythonコード実行環境を定義する*実行構成*を作成します。この場合、デフォルトのPythonパッケージがインストールされたConda環境が自動的に作成されます。
2. 実験で実行するPythonスクリプトファイルと、それを実行する環境を識別する* Script Configuration *を作成します。

> **Note**: 現時点では、環境設定についてあまり心配する必要はありません。後で詳しく説明します。

次のセルはこれらの構成オブジェクトを設定し、実験を送信します。

In [ ]:
import os
import sys
from azureml.core import Experiment, RunConfiguration, ScriptRunConfig
from azureml.widgets import RunDetails

# create a new RunConfig object
experiment_run_config = RunConfiguration()

# Create a script config
src = ScriptRunConfig(source_directory=experiment_folder, 
                      script='diabetes_experiment.py',
                      run_config=experiment_run_config) 

# submit the experiment
experiment = Experiment(workspace = ws, name = 'diabetes-experiment')
run = experiment.submit(config=src)
RunDetails(run).show()
run.wait_for_completion()

以前と同様に、[Azure Machine Learning studio](https://ml.azure.com)のウィジェットまたは実験へのリンクを使用して、実験によって生成された出力を表示できます。また、生成されたメトリックとファイル:

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 実験実行履歴を表示する

同じ実験を複数回実行したので、[Azure Machine Learning studio]（https://ml.azure.com）で履歴を表示し、記録された各実行を調べることができます。または、ワークスペースから名前で実験を取得し、SDKを使用して実行を繰り返すことができます:

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['diabetes-experiment']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

> **詳しくは**: 実験の実行の詳細については、Azure MLドキュメントの[このトピック](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs)を参照してください。実行時にメトリックを記録する方法の詳細については、[このトピック](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments)を参照してください。